<a href="https://colab.research.google.com/github/sethkipsangmutuba/SQL/blob/main/3a_Aggregation_using_window_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Notebook: Aggregation Using Window Functions

In this notebook, we'll explore how to use **SQL window functions** to perform advanced aggregations on the Titanic dataset — all within a Python (Google Colab) environment.

Window functions allow you to:

- Compute running totals, ranks, and averages
- Compare values across rows in the same group
- Perform calculations without collapsing rows (unlike `GROUP BY`)

---

## Setup: Titanic Dataset + SQLite


In [48]:
import pandas as pd
import numpy as np
import sqlite3
import seaborn as sns

# Load Titanic dataset and simulate "UN-like" structure
df = sns.load_dataset("titanic").dropna(subset=["age", "fare", "sex", "class"])

# Simulate columns
np.random.seed(42)
df["time_period"] = 2020
df["sub_region"] = np.random.choice(["Western Europe", "Eastern Europe", "Northern America", "Sub-Saharan Africa"], len(df))
df["country_name"] = df["embarked"].fillna("Unknown") + "_" + df.index.astype(str)
df["land_area"] = np.random.uniform(50, 1000, size=len(df)).round(2)
df["est_population_in_millions"] = (df["age"] + np.random.uniform(0, 5, size=len(df))) / 2
df["est_gdp_in_billions"] = df["fare"] * 0.1
df["pct_managed_drinking_water_services"] = np.random.uniform(30, 100, size=len(df))
df["pct_managed_sanitation_services"] = np.random.uniform(20, 90, size=len(df))

# Create SQLite in-memory DB
conn = sqlite3.connect(":memory:")
df.to_sql("access_to_basic_services", conn, index=False, if_exists="replace")


714

---

## Task 1: Select Data for Year 2020 (No NULL `land_area`)

Filter the dataset using a `WHERE` clause to:

- Select only rows where `year = 2020`
- Exclude rows where `land_area` is `NULL`


In [49]:
query1 = """
SELECT
    sub_region,
    country_name,
    land_area
FROM access_to_basic_services
WHERE time_period = 2020
  AND land_area IS NOT NULL;
"""

pd.read_sql_query(query1, conn)


,sub_region,country_name,land_area
0,Northern America,S_0,606.49
1,Sub-Saharan Africa,C_1,943.22
2,Western Europe,S_2,596.70
3,Northern America,S_3,418.76
4,Northern America,S_4,661.12
...,...,...,...
709,Sub-Saharan Africa,Q_885,276.84
710,Northern America,S_886,154.95
711,Eastern Europe,S_887,386.89
712,Western Europe,C_889,322.88


---

## Task 2: Land Area as % of Sub-region Total (Window Function)

Use a **window function** to calculate each country's land area as a percentage of the total land area in its sub-region.

- Use `SUM(land_area) OVER (PARTITION BY subregion)` to compute the total land area per sub-region
- Then divide each country’s `land_area` by that total


In [50]:
query2 = """
SELECT
    sub_region,
    country_name,
    land_area,
    ROUND(land_area * 100.0 / SUM(land_area) OVER (PARTITION BY sub_region), 4) AS pct_sub_region_land_area
FROM access_to_basic_services
WHERE time_period = 2020
  AND land_area IS NOT NULL;
"""

pd.read_sql_query(query2, conn)


,sub_region,country_name,land_area,pct_sub_region_land_area
0,Eastern Europe,S_10,963.13,1.0693
1,Eastern Europe,S_23,72.11,0.0801
2,Eastern Europe,S_25,317.76,0.3528
3,Eastern Europe,S_33,647.50,0.7188
4,Eastern Europe,C_34,883.60,0.9810
...,...,...,...,...
709,Western Europe,C_866,70.21,0.0797
710,Western Europe,S_877,646.13,0.7339
711,Western Europe,S_880,984.88,1.1186
712,Western Europe,S_881,779.86,0.8858


---

## Task 3: Running Average Population by Sub-region

Use a **window function** to calculate the running average of population within each sub-region, ordered by country name.

- Use `AVG(population) OVER (PARTITION BY subregion ORDER BY country)`  
- This creates a cumulative (running) average of population values per sub-region


In [51]:
query3 = """
SELECT
    sub_region,
    country_name,
    time_period,
    pct_managed_drinking_water_services,
    pct_managed_sanitation_services,
    est_gdp_in_billions,
    est_population_in_millions,
    ROUND(AVG(est_population_in_millions) OVER (
        PARTITION BY sub_region ORDER BY time_period
    ), 4) AS running_average_population
FROM access_to_basic_services
WHERE est_gdp_in_billions IS NOT NULL;
"""

pd.read_sql_query(query3, conn)


,sub_region,country_name,time_period,pct_managed_drinking_water_services,pct_managed_sanitation_services,est_gdp_in_billions,est_population_in_millions,running_average_population
0,Eastern Europe,S_10,2020,32.917103,79.287820,1.67000,3.965863,15.8540
1,Eastern Europe,S_23,2020,84.225747,70.589487,3.55000,15.624117,15.8540
2,Eastern Europe,S_25,2020,42.963690,29.410459,3.13875,20.281721,15.8540
3,Eastern Europe,S_33,2020,76.221190,76.310605,1.05000,33.551216,15.8540
4,Eastern Europe,C_34,2020,83.506086,39.513191,8.21708,14.466096,15.8540
...,...,...,...,...,...,...,...,...
709,Western Europe,C_866,2020,85.335686,83.965191,1.38583,14.720603,16.1003
710,Western Europe,S_877,2020,90.448327,55.219273,0.78958,11.888337,16.1003
711,Western Europe,S_880,2020,56.273094,80.117174,2.60000,13.288517,16.1003
712,Western Europe,S_881,2020,50.004849,67.385855,0.78958,18.220337,16.1003
